In [61]:
import pandas as pd
from sdv.lite import TabularPreset
from sdv.evaluation import evaluate


In [62]:
config = pd.read_json("config.json") #config file(dictionary)
data_file = config["Input File"][0]
print(data_file)
target_file = config["Output File"][0]
data = pd.read_csv(str(data_file))
headers = data.columns

feature_dict = config["Features"]

features = [] #names of relevant headers
for feature in config["Features"].keys():
    features.append(feature) # wanted headers

allowed_percentage = config["Percentage"][0]
number_of_samples = int(config["N_samples"][0])
#print(features)


split1.csv


In [63]:

#Remove unspecified columns
for header in headers:
    if not header in features:
        data.pop(header)
        #print("Deleting column", header, "because of configuration.")

#Remove columns with to many NaNs
headers = data.columns #fixed list of columns we need
for header in headers:
    nan_percentage = data[header].isna().sum()/len(data[header].index)
    if nan_percentage > allowed_percentage:
        data.pop(header)
        headers = headers.drop(header)
        print("Deleting column", header, "because of" ,"{:.2%}".format(nan_percentage), "NaNs.")

#removing unwanted nans and interpolating others

numericals = []
for x in features:
    if feature_dict[x][0] == "numerical":
        numericals.append(x)

for numerical in numericals:
    data[numerical] = pd.to_numeric(data[numerical], errors='coerce')

data.interpolate(method='pad')






Deleting column Neoplasm Histologic Grade because of 91.28% NaNs.
Deleting column Treatment Details before PDX because of 90.58% NaNs.


,Cancer Type Detailed,Mutation Count,Oncotree Code,Overall Survival (Months),Sample Type,Diagnosis Age,TERT,IDH1,PABPC3,PTEN
0,Glioblastoma Multiforme,173.0,GBM,30.00,Recurrence,40.0,Thalidomide,Carboplatin,WT,WT
1,Glioblastoma Multiforme,1756.0,GBM,21.00,Recurrence,67.0,WT,WT,E2101K,P191T I195V
2,Glioblastoma Multiforme,158.0,GBM,8.04,Primary,62.0,WT,WT,K312R T319I,I101T
3,Glioblastoma Multiforme,161.0,GBM,25.92,Primary,64.0,WT,WT,V293G A313V K312R A154G K156R,WT
4,Glioblastoma Multiforme,669.0,GBM,44.88,Recurrence,65.0,WT,WT,WT,A313Rfs*14 F335Lfs*19 M326Gfs*21 K231E P191T E...
...,...,...,...,...,...,...,...,...,...,...
855,Anaplastic Oligodendroglioma,12.0,AODG,12.90,Recurrence,64.0,Promoter,WT,NS,WT
856,Anaplastic Oligoastrocytoma,14.0,AOAST,54.80,Recurrence,30.0,Promoter,R132H,NS,WT
857,Anaplastic Oligodendroglioma,7.0,AODG,93.90,Recurrence,45.0,Promoter,R132H,NS,WT
858,Anaplastic Oligodendroglioma,5.0,AODG,48.30,Recurrence,42.0,Promoter,R132H,NS,WT


In [64]:
#create validation set

validation_set = data.sample(frac=0.3)
validation_set.reset_index()


,index,Cancer Type Detailed,Mutation Count,Oncotree Code,Overall Survival (Months),Sample Type,Diagnosis Age,TERT,IDH1,PABPC3,PTEN
0,251,Glioblastoma Multiforme,3.0,GBM,58.60,Recurrence,45.0,Promoter,WT,NS,WT
1,125,Anaplastic Oligodendroglioma,55.0,AODG,45.70,NaN,54.0,Promoter,R132H,NS,WT
2,362,Glioblastoma Multiforme,6.0,GBM,58.40,Recurrence,54.0,Promoter,WT,NS,I101T
3,532,Oligodendroglioma,3.0,ODG,136.00,Primary,42.0,Promoter,R132H,NS,WT
4,288,Anaplastic Astrocytoma,4.0,AASTR,24.70,Primary,37.0,WT,R132H,NS,WT
...,...,...,...,...,...,...,...,...,...,...,...
253,221,Anaplastic Astrocytoma,5.0,AASTR,66.00,Primary,37.0,WT,R132H,NS,WT
254,9,Glioblastoma Multiforme,120.0,GBM,2.88,Primary,69.0,WT,WT,WT,WT
255,673,Anaplastic Astrocytoma,3.0,AASTR,120.00,Primary,36.0,WT,R132H,NS,WT
256,844,Astrocytoma,15.0,ASTR,137.00,Primary,30.0,WT,R132H,WT,WT


In [65]:
#Generating Metadata

metadata = {}
metadata["fields"] = {}

for x in headers:
    metadata["fields"][x] = {}
    metadata["fields"][x]["type"] = feature_dict[x][0]
    if feature_dict[x][0] == "numerical":
        metadata["fields"][x]["subtype"] = feature_dict[x][1]
metadata["constraints"] = []

#print(metadata)



In [66]:
#fit a model to the data and generating data
#model = TabularPreset(name='FAST_ML', metadata=metadata)
model = TabularPreset(name='FAST_ML', metadata=metadata)

model.fit(data)

new_data = model.sample(num_rows=number_of_samples)

new_data = new_data.round(decimals = 2)

new_data.to_csv(target_file, index=False)

print("The generated data is", "{:.2%}".format(evaluate(new_data, validation_set, metrics=['KSTest'])), "accurate to the original data. If this is unsatisfactory try using less or different features, more data or removing more NaNs.")

The generated data is 78.00% accurate to the original data. If this is unsatisfactory try using less or different features, more data or removing more NaNs.
